In [ ]:
# デモに必要なライブラリーを読み込み
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# csvファイルのデータ型を宣言して読み込み
lrdataschema = StructType([
    StructField("X1", FloatType(), True),
    StructField("X2", FloatType(), True),
    StructField("Label", IntegerType(), True)])
inputData = spark.read.csv("logisticRegressionData.csv", mode="DROPMALFORMED", schema=lrdataschema)

In [ ]:
# NumPy配列に変換してラベルと説明変数に分離
ar = np.array(inputData.rdd.map(lambda row: [row[0], row[1], row[2]]).collect())
L=ar[:,2] ; X1=ar[:,0] ; X2=ar[:,1]

In [ ]:
# データを陽性と陰性に分離してプロット
X1_0 = X1[L==0] ; X2_0 = X2[L==0]
X1_1 = X1[L==1] ; X2_1 = X2[L==1]

%matplotlib inline
plt.axis([-1,1.5,-1,1.5])
plt.plot(X1_0,X2_0,"o",X1_1,X2_1,"x")

In [ ]:
# いくつかのデータをそのまま出力
for i in range(0,120,25):
    print("Label:", L[i], "X1", X1[i], "X2", X2[i])

In [ ]:
# 曲線（円弧）のモデルを得るために説明変数を多項式に展開
list2=[]
for i in range(len(L)):
    list2.append(LabeledPoint(L[i], \
            [X1[i], X2[i], X1[i]**2, X1[i]*X2[i], X2[i]**2,
            X1[i]**3, X1[i]**2 * X2[i], X1[i]**1 * X2[i]**2, X2[i]**3,
            X1[i]**4, X1[i]**3 * X2[i], X1[i]**2 * X2[i]**2, X1[i]**1 * X2[i]**3, X2[i]**4, \
            X1[i]**5, X1[i]**4 * X2[i], X1[i]**3 * X2[i]**2, X1[i]**2 * X2[i]**3, X1[i]**1 * X2[i]**4, X2[i]**5,
            X1[i]**6, X1[i]**5 * X2[i], X1[i]**4 * X2[i]**2, X1[i]**3 * X2[i]**3, X1[i]**2 * X2[i]**4, X1[i]**1 * X2[i]**5, X2[i]**6
            ]))
parsedData = sc.parallelize(list2)

In [ ]:
# データをロジスティックス回帰のライブラリーに投入してモデルの学習を実行
model = LogisticRegressionWithLBFGS.train(parsedData)

In [ ]:
# 学習の結果として得られたモデルのパラメーターを表示
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainAcc = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(parsedData.count())
print("Training Accuracy = " + str(trainAcc))
print(model.weights)

In [ ]:
# モデルを可視化するために、モデルのパラメーターから得られる決定境界を計算
u = np.linspace(-1, 1.5, 50)
v = np.linspace(-1, 1.5, 50)
z = np.zeros(( len(u), len(v) ))

for i in xrange(len(u)):
    for j in xrange(len(v)):
        z[i,j] = np.dot([u[i], v[j], u[i]**2, u[i]*v[j], v[j]**2, \
                         u[i]**3, u[i]**2 * v[j]**1, u[i]**1 * v[j]**2, v[j]**3, \
                         u[i]**4, u[i]**3 * v[j]**1, u[i]**2 * v[j]**2, u[i]**1 * v[j]**3, v[j]**4, \
                         u[i]**5, u[i]**4 * v[j]**1, u[i]**3 * v[j]**2, u[i]**2 * v[j]**3, u[i]**1 * v[j]**4, v[j]**5, \
                         u[i]**6, u[i]**5 * v[j]**1, u[i]**4 * v[j]**2, u[i]**3 * v[j]**3, u[i]**2 * v[j]**4, u[i]**1 * v[j]**5, v[j]**6] ,model.weights)
        
z = np.transpose(z)
# データと決定境界をプロット
plt.axis([-1,1.5,-1,1.5])
plt.plot(X1_0,X2_0,"o",X1_1,X2_1,"x")
plt.contour(u, v, z, levels=[0], linewidth=2).collections[0]